In [1]:
import pandas as pd
import numpy as np
import datetime as dt

import yfinance as yf

import warnings
warnings.filterwarnings("ignore")

In [2]:
# download historical data for required stocks
ticker = 'AAPL'
AAPL = yf.download(ticker, dt.date.today() - dt.timedelta(365 * 5), dt.date.today())

[*********************100%***********************]  1 of 1 completed


In [3]:
AAPL.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-10-31,28.412500,28.557501,28.299999,28.385000,26.620600,105677600
2016-11-01,28.365000,28.442499,27.632500,27.872499,26.139963,175303200
2016-11-02,27.850000,28.087500,27.807501,27.897499,26.163406,113326800
2016-11-03,27.745001,27.865000,27.387501,27.457500,25.882963,107730400
2016-11-04,27.132500,27.562500,27.027500,27.209999,25.649658,123348000


In [4]:
AAPL.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-10-25,148.679993,149.369995,147.619995,148.639999,148.639999,50720600
2021-10-26,149.330002,150.839996,149.009995,149.320007,149.320007,60893400
2021-10-27,149.360001,149.729996,148.490005,148.850006,148.850006,56094900
2021-10-28,149.820007,153.169998,149.720001,152.570007,152.570007,100077900
2021-10-29,147.220001,149.940002,146.410004,149.800003,149.800003,124850400


In [5]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting.test import SMA


Loading BokehJS ...

In [6]:
class SmaCross(SignalStrategy, TrailingStrategy):
    n1 = 50
    n2 = 200
    
    def init(self):
        super().init()
        
        sma1 = self.I(SMA, self.data.Close, self.n1)
        sma2 = self.I(SMA, self.data.Close, self.n2)
        
        # Where sma1 crosses sma2 upwards. Diff gives us [-1, 0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0) # Upwards/long only
        
        entry_size = signal * 0.95
        
        self.set_signal(entry_size = entry_size)
        
        self.set_atr_periods(20)
        self.set_trailing_sl(2) # ATR stop loss

In [7]:
from backtesting import Backtest

bt = Backtest(AAPL, SmaCross, commission=0.002, trade_on_close=True)
bt.run()
bt.plot()

Row(id='1517', ...)

In [8]:
stats = bt.run()
stats

Start                     2016-10-31 00:00:00
End                       2021-10-29 00:00:00
Duration                   1824 days 00:00:00
Exposure Time [%]                    0.317712
Equity Final [$]                  9598.299228
Equity Peak [$]                       10000.0
Return [%]                          -4.017008
Buy & Hold Return [%]              427.743533
Return (Ann.) [%]                   -0.817277
Volatility (Ann.) [%]                 1.35204
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -4.017008
Avg. Drawdown [%]                   -4.017008
Max. Drawdown Duration      906 days 00:00:00
Avg. Drawdown Duration      906 days 00:00:00
# Trades                                    1
Win Rate [%]                              0.0
Best Trade [%]                      -4.249967
Worst Trade [%]                     -4.249967
Avg. Trade [%]                    

In [9]:
# Parameter optimization
stats = bt.optimize(n1=range(5, 50, 5),
                        n2=range(10, 200, 5),
                        maximize='Sharpe Ratio',
                        constraint=lambda param: param.n1 < param.n2)
stats

Start                     2016-10-31 00:00:00
End                       2021-10-29 00:00:00
Duration                   1824 days 00:00:00
Exposure Time [%]                   16.600477
Equity Final [$]                 17984.028582
Equity Peak [$]                  18574.030967
Return [%]                          79.840286
Buy & Hold Return [%]              427.743533
Return (Ann.) [%]                   12.465129
Volatility (Ann.) [%]                9.133864
Sharpe Ratio                         1.364716
Sortino Ratio                        2.867101
Calmar Ratio                         2.014747
Max. Drawdown [%]                   -6.186945
Avg. Drawdown [%]                   -1.903587
Max. Drawdown Duration      315 days 00:00:00
Avg. Drawdown Duration       47 days 00:00:00
# Trades                                    7
Win Rate [%]                            100.0
Best Trade [%]                      26.027287
Worst Trade [%]                      2.923406
Avg. Trade [%]                    

In [10]:
# get the optimization parameters of n1, n2
stats._strategy

<Strategy SmaCross(n1=5,n2=175)>

In [11]:
class SmaCross2(SignalStrategy, TrailingStrategy):
    n1 = 5
    n2 = 175
    
    def init(self):
        super().init()
        
        sma1 = self.I(SMA, self.data.Close, self.n1)
        sma2 = self.I(SMA, self.data.Close, self.n2)
        
        # Where sma1 crosses sma2 upwards. Diff gives us [-1, 0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0) # Upwards/long only
        
        entry_size = signal * 0.95
        
        self.set_signal(entry_size = entry_size)
        
        self.set_atr_periods(20)
        self.set_trailing_sl(2) # ATR stop loss

In [12]:
bt = Backtest(AAPL, SmaCross2, commission=0.002, trade_on_close=True)
bt.run()
bt.plot()

Row(id='2370', ...)

In [13]:
stats = bt.run()
stats

Start                     2016-10-31 00:00:00
End                       2021-10-29 00:00:00
Duration                   1824 days 00:00:00
Exposure Time [%]                   16.600477
Equity Final [$]                 17984.028582
Equity Peak [$]                  18574.030967
Return [%]                          79.840286
Buy & Hold Return [%]              427.743533
Return (Ann.) [%]                   12.465129
Volatility (Ann.) [%]                9.133864
Sharpe Ratio                         1.364716
Sortino Ratio                        2.867101
Calmar Ratio                         2.014747
Max. Drawdown [%]                   -6.186945
Avg. Drawdown [%]                   -1.903587
Max. Drawdown Duration      315 days 00:00:00
Avg. Drawdown Duration       47 days 00:00:00
# Trades                                    7
Win Rate [%]                            100.0
Best Trade [%]                      26.027287
Worst Trade [%]                      2.923406
Avg. Trade [%]                    